# Модуль 5.3 — Тестирование и оценка качества

**Цель:** проверять, что ответы LLM стабильно соответствуют ожиданиям.

**Что сделаем:**
- напишем мини‑тесты на цепочку
- проверим «золотой набор»
- посчитаем простую метрику качества

In [ ]:
%pip -q install -U \
  langchain \
  langchain-community \
  pydantic==2.12.3 \
  requests==2.32.4

## 1) Мини‑тест на цепочку (без реального API)

Используем FakeListLLM — он возвращает заранее заданные ответы.

In [ ]:
from langchain_community.llms import FakeListLLM
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

fake = FakeListLLM(responses=["Кэш — это память для быстрых повторов."])

prompt = PromptTemplate.from_template("Объясни, что такое кэш одним предложением.")
chain = prompt | fake | StrOutputParser()

result = chain.invoke({})
assert "Кэш" in result
print("Тест пройден:", result)

## 2) «Золотой набор» (golden set)

Мини‑таблица вопросов и ожидаемых ответов.

In [ ]:
golden = [
    ("Что такое кэш?", "память"),
    ("Что такое ретривер?", "поиск"),
    ("Что такое RAG?", "контекст"),
]

# В тестах мы не зовем реальную модель — используем заглушку
fake = FakeListLLM(responses=[
    "Кэш — это память для быстрых повторов.",
    "Ретривер — это поиск по базе.",
    "RAG — это ответы с контекстом.",
])

test_prompt = PromptTemplate.from_template("{q}")
chain = test_prompt | fake | StrOutputParser()

hits = 0
for q, expected_keyword in golden:
    ans = chain.invoke({"q": q})
    ok = expected_keyword in ans.lower()
    hits += 1 if ok else 0
    print(q, "->", ans, "| ok:", ok)

accuracy = hits / len(golden)
print("Accuracy:", accuracy)

## 3) Мини‑регрессия

Если упадёт хоть один тест — значит, что-то изменилось в логике.

In [ ]:
assert accuracy >= 0.66, "Слишком низкое качество: проверь цепочку"
print("Регрессия пройдена")